In [84]:
import requests
import zipfile
import io
import yaml

with open("config.yaml") as file:
    yamlFile = yaml.safe_load(file)

url = "http://oasis.caiso.com/oasisapi/SingleZip"
extract_directory = ""
params = {
    "resultformat": "6",
    "queryname": yamlFile["market"],
    "market_run_id": yamlFile["market_run_id"],
    "execution_type": yamlFile["execution_type"],
    "version": "1",
    "startdatetime": "20240218T08:00-0000",
    "enddatetime": "20240219T08:00-0000"
}

response = requests.get(url, params=params)


In [85]:

if response.status_code == 200:
    data = response.content

    zip_data = io.BytesIO(data)
    csv = None
    with zipfile.ZipFile(zip_data, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        print(file_list)
        for file_name in file_list:
            csv = file_name[:-25]+".csv"
            file_data = zip_ref.read(file_name)
            with open(file_name[:-25]+".csv", 'wb') as file:
                file.write(file_data)
            
else:
    print("Request failed with status code:", response.status_code)

['20240218_20240219_SLD_REN_FCST_RTPD_20240308_08_01_50_v1.csv']


In [105]:
import pandas as pd
import numpy as np

data = pd.read_csv(csv)
# data = np.array(data)
print(data.shape)
print(data.columns.tolist())

zone_id = yamlFile["zone_id"]
data_query = data.query(f'TRADING_HUB == "{zone_id}"')
print(data_query.shape)

(576, 14)
['OPR_DT', 'OPR_HR', 'OPR_INTERVAL', 'INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT', 'TRADING_HUB', 'RENEWABLE_TYPE', 'LABEL', 'XML_DATA_ITEM', 'MARKET_RUN_ID_POS', 'RENEW_POS', 'MW', 'MARKET_RUN_ID', 'GROUP']
(192, 14)
